In [18]:
import pandas as pd
from resources.connection import Conn
PATH_DATASET = "resources/datatran2022.csv"
SCHEMA = "datatran"


EXTRACT

In [10]:
def createDataFrame():    
    df = pd.read_csv(PATH_DATASET,sep=";",encoding="cp1252") 
    return df

TRANSFORM

In [16]:
def changeDataFrame(df):
    df['br'] = df['br'].fillna(0)
    df['br'] = df['br'].astype('int64')
    notUsedCols = ["km",
                   "municipio",
                   "fase_dia",
                   "sentido_via",
                   "tipo_pista",
                   "tracado_via",
                   "uso_solo",
                   "pessoas",
                   "feridos",
                   "latitude",
                   "longitude",
                   "regional",
                   "delegacia",
                   "uop"]
    return df.drop(columns=notUsedCols)

LOAD

In [ ]:
def loadTables(df):
    #laod Auxiliaries Tables
    dictAux = {'dia_semana':'dias',
               'uf':'estado',
               'causa_acidente':'causas',
               'tipo_acidente':'tipos',
               'classificacao_acidente':'classificacoes',
               'condicao_metereologica':'condicoes'}
    
    for col, table in dictAux.items():
        df[[col]].to_sql(table,con=Conn(),schema=SCHEMA,if_exists='append')


    #laod Accidents Table
    tableAcc = 'acidentes'
    usedColsAcc = ['id','data_inversa','horario','br','mortos', 'feridos_leves', 'feridos_graves','ilesos', 'ignorados', 'veiculos']
    df[usedColsAcc].to_sql(tableAcc,con=Conn(),schema=SCHEMA,if_exists='append')

MAIN

In [17]:
def normalize():
    #build general DataFrame
    dfDatatran = createDataFrame()

    #modify general DataFrame
    dfDatatran = changeDataFrame(dfDatatran)

    #load Tables
    loadTables(dfDatatran)